In [ ]:
%pip install ultralytics

In [ ]:
%run ./common

In [ ]:
test_image=next(test_images())
print(f"test_image:{test_image.name}")
image_base64 = image_to_base64(test_image, verify=True)
input_example = pd.DataFrame({"image_base64": [image_base64]})

In [ ]:
from mlflow.models.signature import ModelSignature
from mlflow.types import Schema, ColSpec, DataType


import io
import base64
import pandas as pd
from PIL import Image
import mlflow


input_schema = Schema([ColSpec(DataType.string, "image_base64")])
output_schema = Schema([ColSpec(DataType.string, "json_result")])

signature = ModelSignature(
    inputs=input_schema,
    outputs=output_schema,
)

input_example = pd.DataFrame({"image_base64": [image_base64]})

with mlflow.start_run() as run:
    mlflow.pyfunc.log_model(
        artifact_path="model",
        python_model="yolo_model.py",
        artifacts={
            "weights": "yolo11n.pt",
        },
        pip_requirements=["ultralytics"],
        input_example=input_example,
        signature=signature,
    )

model_uri = f"runs:/{run.info.run_id}/model"

In [ ]:
from mlflow.tracking import MlflowClient

mv = mlflow.register_model(
    model_uri=model_uri,
    name=REGISTERED_MODEL_NAME,
)

MlflowClient().set_registered_model_alias(
    name=REGISTERED_MODEL_NAME,
    alias=ALIAS,
    version=mv.version,
)

print(mv.version)

In [ ]:
import base64
import json
import cv2
import numpy as np
import pandas as pd
import mlflow
from IPython.display import display_html


model_uri = f"models:/{REGISTERED_MODEL_NAME}@{ALIAS}"
model = mlflow.pyfunc.load_model(model_uri)

cards = []

for test_image in test_images():
    image_base64 = image_to_base64(test_image)

    df = pd.DataFrame({"image_base64": [image_base64]})
    result = model.predict(df)
    predictions = result.iloc[0]["detections"]
    annotated_test_image=annotate_image(test_image, predictions)
    annotated_b64 = image_to_base64(annotated_test_image)
    

    boxes_json = json.dumps(predictions, indent=2, sort_keys=True)

    cards.append(
        f"""
        <div style="border:1px solid #ddd; padding:8px; display:flex; flex-direction:column; gap:8px;">
          <div style="font-size:12px; font-weight:600;">{test_image.name}</div>
          <img src="data:image/png;base64,{annotated_b64}" style="max-width:100%;"/>
          <pre style="
            font-size:11px;
            line-height:1.3;
            background:#f7f7f7;
            padding:6px;
            overflow:auto;
            max-height:240px;
          ">{boxes_json}</pre>
        </div>
        """
    )

html = f"""
<div style="
  display:grid;
  grid-template-columns:repeat(auto-fill, minmax(360px, 1fr));
  gap:12px;
">
  {''.join(cards)}
</div>
"""

displayHTML(html)